In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import perceval as pcvl
import lo_reservoir as lrs

In [2]:
m = 5
t_max = 10000
simulator = lrs.PhotonicReservoirSimulator(m, t_max, overlapping=True)

In [3]:
simulator.set_circuit_with_stored_layers()

Generating layers: 100%|██████████| 10000/10000 [00:14<00:00, 669.87it/s]


In [5]:
num_layers = 10000
simulator.set_circuit_with_stored_layers(num_layers=num_layers)
data = simulator.generate_rndm_param_matrix(num_layers)
simulator.set_circuit_parameters(data)

input_state = pcvl.BasicState([3]*(m-2) + [0]*2)
expectation_values = simulator.calculate_mode_expectations(input_state)
print("Expectation values for each mode:", expectation_values)

Expectation values for each mode: [1.5636030641942613, 1.451970949105451, 1.53791698061011, 1.6842319069269347, 2.7622770991691734]


# Generate matrix (t_max, m)

In [7]:
from tqdm import tqdm

import numpy as np

def sequential_expectation_calculation(simulator, data):
    # Initialize an empty matrix to store expectation values
    expectations_matrix = []

    for t in tqdm(range(simulator.t_max), desc="Processing time steps"):
        # Generate a circuit with num_layers = t
        simulator.set_circuit_with_stored_layers(num_layers=t)

        # Take the first t rows of the data matrix
        params_subset = data[:t]

        # Set the parameters of the circuit
        simulator.set_circuit_parameters(params_subset)

        # Evaluate the expectation values
        expectation_values = simulator.calculate_mode_expectations(input_state)

        # Append the expectation values to the matrix
        expectations_matrix.append(expectation_values)

    return expectations_matrix

# Usage
m = 5
t_max = 100
simulator = lrs.PhotonicReservoirSimulator(m, t_max, overlapping=True)
_ = simulator.set_circuit_with_stored_layers()
data = simulator.generate_rndm_param_matrix(t_max)  # Assuming this generates a matrix of shape (t_max, (m-1)*5)

input_state = pcvl.BasicState([3]*(m-2) + [0]*2)
expectations_matrix = sequential_expectation_calculation(simulator, data)

# Verify the shape of the resulting matrix
print("Shape of the expectations matrix:", np.array(expectations_matrix).shape)


Processing time steps: 100%|██████████| 100/100 [00:01<00:00, 51.01it/s]

Shape of the expectations matrix: (100, 5)


In [8]:
print("Expectation matrix:", expectations_matrix)

Expectation matrix: [[3.0, 3.0, 3.0, 0.0, 0.0], [3.000000000000001, 2.9999999999999964, 0.6484518284574089, 2.283325959382962, 0.06822221215962397], [3.0000000000000036, 2.884939492400793, 2.6908632836971456, 0.34527612724698264, 0.07892109665508568], [2.9100688313344065, 2.7834983030956137, 2.877176506969766, 0.35171855324501483, 0.07753780535522332], [2.881554079822037, 2.817167251645197, 1.76974890930753, 1.1623237963307618, 0.36920596289448326], [2.82081699253112, 1.9817023194570786, 1.1022859864266996, 2.754279510431738, 0.3409151911533866], [2.7696912585525015, 0.3810950994754944, 2.8869709233263317, 0.14704129824774875, 2.815201420397924], [0.7353520683784659, 2.692552122234257, 0.339542643864131, 2.8649374295134904, 2.3676157360096552], [2.4861015078226165, 1.017116360123606, 0.6937390497970304, 2.405361369231892, 2.397681713024848], [0.9368574671029378, 1.0553535176443058, 2.045219501479638, 2.3348091952983285, 2.6277603184747775], [0.9324553382966565, 0.9646091785242619, 2.12